In [12]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)

mnist_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5),(1))
])

train_data = datasets.MNIST(root='/home/gus/PyTorch',
                           train=True,
                           download=True,with torch.no_grad():
    for X,Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        prediction = model(X)
        correct_prediction = torch.argmax(prediction,1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:',accuracy.item())

                           transform=mnist_transform)
traindata = DataLoader(train_data,
                      batch_size=32,
                      shuffle=True)
test_data = datasets.MNIST(root='/home/gus/PyTorch',
                          train=False,
                          download=True,
                          transform=mnist_transform)
testdata = DataLoader(test_data,
                     batch_size=32,
                     shuffle=True)
lr = 0.0005

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5,self).__init__()
        # input shape = (?, 1, 32, 32)
        # Conv -> (?, 6, 28, 28)
        # Pool -> (?, 6, 14, 14)
        self.layer1 = nn.Sequential(                   
            torch.nn.Conv2d(1,6,5,stride=1,padding=0),#C1 layer using 5x5 filters
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=2,stride=2) #S2 layer
        )
        # input shape = (?, 6, 14, 14)
        # Conv -> (?, 16, 10, 10)
        # Pool -> (?, 16, 5, 5)
        self.layer2 = nn.Sequential(
            torch.nn.Conv2d(6,16,5,stride=1,padding=0), #C3 layer, couldn't implement exactly
            torch.nn.Tanh(),
            torch.nn.AvgPool2d(kernel_size=2,stride=2)  #S4 layer
        )
        # input shape = (?, 16, 5, 5)
        # Conv -> (?, 120, 1, 1)
        self.layer3 = nn.Sequential(
            torch.nn.Conv2d(16,120,5,stride=1,padding=0), #C5 layer
            torch.nn.Tanh()
        )
        # input shape = (32[batch_size], 120, 1, 1) -> .view
        self.layer4 = nn.Sequential(
            torch.nn.Linear(120,84,bias=True), #F6 layer
            torch.nn.Tanh(),
            torch.nn.Linear(84,10,bias=True) 
        )

        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1) # C5 layer
        out = self.layer4(out)
        return out
    
model=LeNet5().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = lr)

total_batch = len(traindata)

for epoch in range(15):
    avg_cost = 0
    for X,Y in traindata:     
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis,Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/15
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))


[Epoch:    1] cost = 54.5152283
[Epoch:    2] cost = 17.5967007
[Epoch:    3] cost = 12.429245
[Epoch:    4] cost = 9.34422398
[Epoch:    5] cost = 7.50939989
[Epoch:    6] cost = 6.07971621
[Epoch:    7] cost = 5.18916273
[Epoch:    8] cost = 4.39038515
[Epoch:    9] cost = 3.61523223
[Epoch:   10] cost = 3.21341014
[Epoch:   11] cost = 2.71466732
[Epoch:   12] cost = 2.33565331
[Epoch:   13] cost = 2.12663794
[Epoch:   14] cost = 1.75133789
[Epoch:   15] cost = 1.73508453


In [14]:
with torch.no_grad():
    a=0
    for X,Y in testdata:
        X = X.to(device)
        Y = Y.to(device)
        
        prediction = model(X)
        correct_prediction = torch.argmax(prediction,1) == Y
        accuracy = correct_prediction.float().mean()
        print('Accuracy:',accuracy.item())
        a +=1
        if a == 10:
            break


Accuracy: 0.96875
Accuracy: 0.96875
Accuracy: 0.96875
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 1.0
Accuracy: 0.96875
Accuracy: 0.96875
Accuracy: 1.0
